# Generate features for houses using an Autoencoder model

In [ ]:
import sys  
sys.path.insert(0, '../visual_home_finder')

import imp
import config, paths, utilities

imp.reload(config)
imp.reload(utilities)

import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from matplotlib import pyplot as plt

from tensorflow.keras.preprocessing import image

In [ ]:
# Find all the csvs in the folder
#list_of_csvs = paths.list_files(config.LISTINGS_PATH, validExts='.csv')
list_of_csvs = [os.path.sep.join([config.LISTINGS_PATH,'Zip_98105.csv']), 
                os.path.sep.join([config.LISTINGS_PATH,'Zip_98115.csv']),
                os.path.sep.join([config.LISTINGS_PATH,'Zip_98117.csv']),
                os.path.sep.join([config.LISTINGS_PATH,'Zip_98107.csv']),
                os.path.sep.join([config.LISTINGS_PATH,'Zip_98103.csv'])]

In [ ]:
# Load the autoencoder model
model_name= "autoencoder_2.h5"
my_model = load_model(os.path.sep.join([config.MODEL_PATH, model_name]))

# Get home-style features from the model
home_feature_model = Model(inputs=my_model.input,
                           outputs=my_model.get_layer('max_pooling2d_7').output)

In [ ]:
my_model.summary()

In [ ]:
# Got thru each of the listings and get appropriate images
home_listings_df = pd.DataFrame()

count = 0
for listings_file in list_of_csvs:
    
    print('Processing file %s...'%(listings_file))
    
    listings_df = pd.read_csv(listings_file)
    index_column = listings_df['ZIP OR POSTAL CODE'].astype(str)+'_'+ (listings_df.index + 1).astype(str)
    listings_df = listings_df.set_index(index_column)
    
    listings_df['home_feature'] = ""
    listings_df.home_feature = listings_df.home_feature.astype('object')
    
    for ii in index_column:
    
        image_name = os.path.sep.join([config.LISTINGS_PATH, ii+'.jpg'])        
        try:
            # Get the home style prediction
            home_features = utilities.get_features_for_image_with_scaling(image_name, home_feature_model)
        except:
            # Remove the particular listing for which image cannot be found
            listings_df = listings_df[listings_df.index != ii]
            print('File %s not found'%image_name)
            continue
        listings_df.at[ii, 'home_feature'] = home_features
    home_listings_df = home_listings_df.append(listings_df)
print('Processing complete!')

In [ ]:
home_listings_df.to_csv(os.path.sep.join([config.FEATURE_PATH, 'home_features_autoencoder.csv']))